## 问答题
1. 支持向量机的基本思想是什么？
划分一个超平面，用来区分样本
2. 什么是支持向量？
在划分边界最近的关键样本数据
3. 在使用 SVM 时，缩放输入值为什么很重要？
SVM计算距离，容易受异常值影响，使用特征缩放（归一化、标准化）
4. SVM 分类器在对实例进行分类时能输出置信度分数吗？概率呢？
置信度分数：SVM 可以通过 decision_function 输出样本到超平面的符号距离（绝对值越大，置信度越高）。
概率：通过设置 probability=True，SVM 可使用 Platt Scaling（逻辑回归校准）输出类别概率（但计算开销较大）。
5. 你如何在 LinearSVC、SVC 和 SGDClassifier 之间进行选择？
LinearSVC：适合大规模线性分类，支持 L2 正则化。
SVC：支持任意核函数（如 RBF），适合非线性分类，计算复杂度高。
SGDClassifier：适合超大规模数据（增量学习）
6. 假设你已经使用 RBF 核训练了一个 SVM 分类器，但它似乎欠拟合训练集。
   你应该增大还是减小 γ（gamma）？C 呢？
γ（gamma）：控制核函数的灵敏度。增大 γ 会使核函数更关注邻近样本，模型更复杂（减小可能欠拟合）。
C：正则化参数。增大 C 会减少误分类惩罚，允许模型更拟合训练数据（减小可能欠拟合）。
调整方向：同时增大 γ（使决策边界更复杂）和增大 C（减少正则化）。
7. ε 不敏感模型是什么意思？
对于损失函数，只有预测值与真实值偏差超过 ε 时才计算误差
8. 使用核技巧有什么意义？
核技巧通过将数据隐式映射到高维空间，使原本线性不可分的数据在高维中线性可分，而无需显式计算高维坐标。解决非线性问题同时计算高效


## 编程题
1. 在葡萄酒数据集上训练SVM分类器，可以使用sklearn.datasets.load_wine()加载它。该数据集包含3个不同种植者生产的178个葡萄酒样本的化学分析：目标是训练一个分类模型，该模型能够根据葡萄酒的化学分析预测种植者。由于SVM分类器是二元分类器，将需要使用“一对全部”对所有三个类进行分类。能达到的精度是多少？

   "一对全部"可以复习 **8_sklearn做分类.ipynb**里的笔记，里面提到了用二元分类器做多分类问题

---

2. 提前预习 **10_支持向量机.ipynb** 最新更新的笔记 （把SVM分类用梯度下降实现）； 大概理解笔记后，尝试自己对照笔记 实现用梯度下降实现SVM分类

   并把自定义的SVM分类用于 iris data(鸢尾花数据)； 取花瓣长度 和 花瓣宽度特征， 分类 看是不是 分类2的花 （(iris.target == 2)

   对比下sklearn自带的SVM分类 和 自定义SVM分类 实现的分类效果

---

3. 在加州房屋数据集上训练和微调SVM回归器。可以使用原始数据集而不是 在课上使用的调整后的版本，
可以使用sklearn.datasets.fetch_california_housing()加载它。目标代表了数十万美元。
由于有超过20000个实例，SVM可能会很慢，因此对于超参数调整，应该使用更少的实例（例如2000个）来测试更多的超参数组合。最佳模型的RMSE是多少？



In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data_wine = datasets.load_wine()
datadf = pd.DataFrame(data_wine.data, columns=data_wine.feature_names)
X_train, X_test, y_train, y_test = train_test_split(datadf, data_wine['target'],random_state=42)


In [2]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier  # 一对其余的分类器
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
ovr_clf = make_pipeline(StandardScaler(), OneVsRestClassifier(SVC(random_state=42)))
ovr_clf.fit(X_train, y_train)
y_pred = ovr_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9777777777777777


In [3]:
from sklearn.base import BaseEstimator
import numpy as np

class MyLinearSVC(BaseEstimator):
    def __init__(self, C, eta0, n_epochs=1000, random_state=None):
        self.C = C
        self._alpha = 1 / (2*C)
        self.eta0 = eta0
        self.n_epochs = n_epochs
        self.random_state = random_state

    @property

    def eta(self):
        return self.eta0

    def fit(self, X, y):
        m = len(X)
        # 随机初始化，可以联系到为什么很多类 会有 random_state这个参数
        if self.random_state:
            np.random.seed(self.random_state)
        w = np.random.randn(X.shape[1], 1)  # n 个 特征权重
        b = 0

        t = np.array(y, dtype=np.float64).reshape(-1, 1) * 2 - 1  # 把 分类 0 和 分类 1 转成  分类-1 和分类 1
        self.Js = []   # 损失值列表

        # 训练
        for epoch in range(self.n_epochs):
            support_vectors_idx = ((X@w + b)*t < 1).ravel()   # 找出支持向量 （损失值不为0的训练集实例， 当前街道内部/分类错误的训练集实例）
            X_sv = X[support_vectors_idx]
            t_sv = t[support_vectors_idx]

            J = (np.sum(w * w) * self._alpha +  np.sum(1- t_sv * (X_sv@w + b))) / m
            self.Js.append(J)

            w_gradient_vector = (2*self._alpha*w  - X_sv.T @ t_sv) / m
            b_derivative = - np.sum(t_sv) / m

            w = w - self.eta * w_gradient_vector
            b = b - self.eta * b_derivative


        self.intercept_ = b
        self.coef_ = w
        support_vectors_idx = ((X@w + b)*t < 1).ravel()
        self.support_vectors_ = X[support_vectors_idx]
        return self

    def decision_function(self, X):
        return X.dot(self.coef_) + self.intercept_

    def predict(self, X):
        return self.decision_function(X) >= 0

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score

data_iris = datasets.load_iris(as_frame=True)
X = data_iris.data[["petal width (cm)", "petal length (cm)"]].values
y = (data_iris.target == 2).values
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X, y,random_state=42)
my_linearSVC = make_pipeline(StandardScaler(), MyLinearSVC(C=1, eta0 =0.1, n_epochs=500, random_state=42))
my_linearSVC.fit(X_train_i, y_train_i)
y_test_i
y_pred_mlsvc = my_linearSVC.predict(X_test_i)
linearSVC = make_pipeline(StandardScaler(),SVC(kernel= 'linear'))
linearSVC.fit(X_train_i,y_train_i)
y_pred_lsvc = linearSVC.predict(X_test_i)
print("accuracy_score y_pred_mlsvc:",accuracy_score(y_pred_mlsvc,y_test_i))
print("accuracy_score y_pred_lsvc:",accuracy_score(y_pred_lsvc,y_test_i))
print("f1_score y_pred_mlsvc:",f1_score(y_test_i,y_pred_lsvc))
print("f1_score y_pred_lsvc:",f1_score(y_test_i,y_pred_lsvc))

accuracy_score y_pred_mlsvc: 1.0
accuracy_score y_pred_lsvc: 1.0
f1_score y_pred_mlsvc: 1.0
f1_score y_pred_lsvc: 1.0


In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

data = fetch_california_housing()
X, y = data.data, data.target

X_subset, _, y_subset, _ = train_test_split(X, y, train_size=2000, random_state=42)

scaler = StandardScaler()
X_subset_scaled = scaler.fit_transform(X_subset)

#
param_grid = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

# 初始化 GridSearchCV
grid_search = GridSearchCV(
    SVR(),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error'
)

grid_search.fit(X_subset_scaled, y_subset)

print("Best Parameters:", grid_search.best_params_)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

best_svr = SVR(**grid_search.best_params_)
best_svr.fit(X_train_scaled, y_train)

y_pred = best_svr.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE",rmse)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
RMSE 0.5678444859569218
